In [1]:
from ryn.common import helper
helper.notebook()

%load_ext autoreload
%autoreload 2

changing directory


# KGC HPO and data instrospection

Sanity checks and somewhat documenting the whole pipeline of kgc training

In [2]:
import ryn
split_ds_path = ryn.ENV.SPLIT_DIR / 'oke.fb15k237'

In [3]:
from ryn.kgc import keen
from ryn.graphs import split

split_ds = split.Dataset.load(
    path=split_ds_path, )

# keen.Dataset.create.invalidate()
keen_ds = keen.Dataset.create(
    name=split_ds.name,
    path=split_ds.path,
    split_dataset=split_ds, )

print(f'{keen_ds}')

ryn pykeen dataset
oke.fb15k237

  training triples factory:
    entities: 14505
    relations: 237
    triples: 272115

  validation triples factory:
    entities: 9799
    relations: 223
    triples: 17526

  testing triples factory:
    entities: 10317
    relations: 224
    triples: 20438



In [4]:
# Each keen.Dataset offers access to three
# pykeen TripleFactories: training, validation and testing

# The strings are prefixed by the ryn id
# (otherwise they would not have been unique)

keen_ds.training.triples[1020:1030]

array([['9148:Enemy of the State', '86:/film/film/production_companies',
        '7699:Touchstone Pictures'],
       ['5886:will.i.am', '43:/people/person/gender',
        '141:male organism'],
       ['2012:1988 Summer Olympics', '65:/olympics/olympic_games/sports',
        '4606:dressage'],
       ['6249:Lawless',
        '13:/film/film/release_date_s./film/film_regional_release_date/film_release_region',
        '1163:Greece'],
       ['1330:Master of Arts',
        '89:/education/educational_degree/people_with_this_degree./education/education/institution',
        '1640:Emmanuel College'],
       ["2585:Olivia d'Abo",
        '232:/film/actor/dubbing_performances./film/dubbing_performance/language',
        '160:English'],
       ['4538:Cole Porter',
        '85:/people/person/spouse_s./people/marriage/type_of_union',
        '434:marriage'],
       ['2793:Victor Garber',
        '45:/base/popstra/celebrity/friendship./base/popstra/friendship/participant',
        '5837:Jennifer Ga

In [5]:
# sanity checking that these triples exist in the original graph

selection = keen_ds.training.triples[1000]
print(f'{selection=}')

# extract ids
h, r, t = map(int, map(lambda s: s.split(':', maxsplit=1)[0], selection))
print(f'{h=}, {r=}, {t=}')

# look in the import source of the original graph
print(f'{split_ds.g.source.ents[h]=}')
print(f'{split_ds.g.source.ents[t]=}')
print(f'{split_ds.g.source.rels[r]=}')

print(f'{(h, t, r) in split_ds.g.source.triples=}')
print(f'{split_ds.g.nx.edges[h, t, r]}')

# alright this looks fine

selection=array(['5470:Polydor Records', '76:/music/record_label/artist',
       '1952:Pete Townshend'], dtype='<U150')
h=5470, r=76, t=1952
split_ds.g.source.ents[h]='Polydor Records'
split_ds.g.source.ents[t]='Pete Townshend'
split_ds.g.source.rels[r]='/music/record_label/artist'
(h, t, r) in split_ds.g.source.triples=True
{'label': '/music/record_label/artist', 'rid': 76}


In [7]:
from ryn.kgc import config
from ryn.kgc import pipeline
from dataclasses import asdict

model_name = 'DistMult'

config = config.Config(
    general=config.General(
        dataset=split_ds.name,
    ),
    tracker=config.Tracker(
        cls='wandb',
        project='ryn-keen-proto',
        experiment=f'{keen_ds.name}-{model_name}',
        reinit=True,
    ),
    model=config.Model(
        cls=model_name,
        embedding_dim=300,
        preferred_device='cuda',
        automatic_memory_optimization=False,
    ),
    optimizer=config.Optimizer(
        cls='Adagrad',
        lr=0.01,
    ),
    regularizer=config.Regularizer(
        cls='LpRegularizer',
        weight=0.1,
        p=2.0,
        normalize=True,
    ),
    loss=config.Loss(
        cls='MarginRankingLoss',
        margin=1.0,
        reduction='mean',
    ),
    evaluator=config.Evaluator(
        cls='RankBasedEvaluator',
        batch_size=200,
    ),
    stopper=config.Stopper(
        cls='EarlyStopper',
        frequency=4,
        patience=100,
        relative_delta=0.0001,
    ),
    training_loop=config.TrainingLoop(
        cls='SLCWATrainingLoop',
    ),
    sampler=config.Sampler(
        cls='BasicNegativeSampler',
        num_negs_per_pos=5,
    ),
    training=config.Training(
        num_epochs=120,
        batch_size=128,
    ),
)

# --

pipeline.single(
    config=config,
    split_dataset=split_ds,
    keen_dataset=keen_ds, )

wandb: Currently logged in as: kantholtz (use `wandb login --relogin` to force relogin)


KeyboardInterrupt: 